In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install keras4torch
!pip install einops

import os
os.chdir('/content/drive/MyDrive/ECML2/Stab/5805_s')

import keras4torch
from   keras4torch.callbacks  import ModelCheckpoint,LRScheduler
import torch
import torch.nn    as nn
import torch.optim as optim
import  torch.nn.functional as     F
import numpy       as np
import pandas      as pd
from copy import deepcopy
import  matplotlib.pyplot   as     plt
from    sklearn.preprocessing import StandardScaler, QuantileTransformer
from    datetime import datetime
import  gc
import STab
from STab import MyClassLoss,CatMap,Num_Cat
from   STab import mainmodel, LWTA, Gsoftmax
MainModel=mainmodel.MainModel

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Volvo')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
Train     = pd.read_csv('train_gen1.csv')
Variants  = pd.read_csv('variants.csv')
Out       = pd.read_csv('public_X_test.csv')
Variants.set_index('ChassisId_encoded',inplace=True)

In [4]:
os.chdir('/content/drive/MyDrive/VolvoFinal')

In [5]:
Train['Tss']  = Train.Timesteps-Train.ChassisId_encoded.map(  Train.groupby('ChassisId_encoded').first().Timesteps)
#Out['Tss']    = Out.Timesteps-Out.ChassisId_encoded.map(  Out.groupby('ChassisId_encoded').first().Timesteps)


In [6]:
Local  = pd.read_csv('OutLocal.csv',index_col='Unnamed: 0')
Global = pd.read_csv('OutGlobal.csv',index_col='Unnamed: 0')

In [7]:

PredOut=pd.DataFrame()
PredOut['Lscore']=Local.mean(axis=1)
PredOut['Gscore']=Global.mean(axis=1)


PredOut['gen']=Out.gen
PredOut['ChassisId']=Out.ChassisId_encoded
PredOut['Timesteps']=Out.Timesteps
PredOut['MinLscore']=PredOut.ChassisId.map( PredOut.groupby('ChassisId').min().Lscore)
PredOut['MinGscore']=PredOut.ChassisId.map( PredOut.groupby('ChassisId').min().Gscore)
PredOut['MeanGscore']=PredOut.ChassisId.map( PredOut.groupby('ChassisId').Gscore.mean())

PredOut['MaxGscore']=PredOut.ChassisId.map( PredOut.groupby('ChassisId').max().Gscore)

PredOut['DGscore']=PredOut.ChassisId.map( PredOut.groupby('ChassisId').first().Gscore) - PredOut.ChassisId.map( PredOut.groupby('ChassisId').last().Gscore)

PredOut['O']=PredOut.Timesteps-  PredOut.ChassisId.map( PredOut.groupby('ChassisId').min().Timesteps)


In [8]:


PredOut.loc[PredOut.O==0, 'final']='Medium'
PredOut.loc[PredOut.O==1, 'final']='Medium'
PredOut.loc[PredOut.O==2, 'final']='Medium'



PredOut.loc[PredOut.O==3, 'final']='High'
PredOut.loc[PredOut.O==4, 'final']='High'
PredOut.loc[PredOut.O==5, 'final']='High'
PredOut.loc[PredOut.O==6, 'final']='High'
PredOut.loc[PredOut.O==7, 'final']='High'
PredOut.loc[PredOut.O==8, 'final']='High'
PredOut.loc[PredOut.O==9, 'final']='High'


In [9]:
Gen1 = PredOut[PredOut['gen']=='gen1']
Gen2 = PredOut[PredOut['gen']=='gen2']

In [10]:


Crit = Gen1[(Gen1.final!='Low') ].MinLscore

PredOut.loc[ (PredOut.gen=='gen1') &(PredOut.O==1) & (PredOut.MinLscore<Crit.quantile(0.1)), 'final']='High'
PredOut.loc[ (PredOut.gen=='gen1') &(PredOut.O==2) & (PredOut.MinLscore<Crit.quantile(0.2)), 'final']='High'


PredOut.loc[ (PredOut.gen=='gen1') &(PredOut.O==3) & (PredOut.MinLscore>Crit.quantile(0.95)), 'final']='Medium'
PredOut.loc[ (PredOut.gen=='gen1') &(PredOut.O==4) & (PredOut.MinLscore>Crit.quantile(0.98)), 'final']='Medium'
PredOut.loc[ (PredOut.gen=='gen1') &(PredOut.O==5) & (PredOut.MinLscore>Crit.quantile(0.99)), 'final']='Medium'


Crit = Gen2[(Gen2.final!='Low') ].MinLscore

PredOut.loc[ (PredOut.gen=='gen2') &(PredOut.O==1) & (PredOut.MinLscore<Crit.quantile(0.1)), 'final']='High'
PredOut.loc[ (PredOut.gen=='gen2') &(PredOut.O==2) & (PredOut.MinLscore<Crit.quantile(0.2)), 'final']='High'


PredOut.loc[ (PredOut.gen=='gen2') &(PredOut.O==3) & (PredOut.MinLscore>Crit.quantile(0.95)), 'final']='Medium'
PredOut.loc[ (PredOut.gen=='gen2') &(PredOut.O==4) & (PredOut.MinLscore>Crit.quantile(0.98)), 'final']='Medium'
PredOut.loc[ (PredOut.gen=='gen2') &(PredOut.O==5) & (PredOut.MinLscore>Crit.quantile(0.99)), 'final']='Medium'

PredOut.loc[ (PredOut.gen=='gen2') & (PredOut.MeanGscore<0.65)  & (PredOut.MaxGscore<0.7) & (PredOut.MinGscore<0.55), 'final']='Low'
PredOut.loc[(PredOut.gen=='gen1') & (PredOut.MeanGscore<0.65 )& (PredOut.MaxGscore<0.85) & (PredOut.MinGscore<0.55), 'final']='Low'

In [11]:

Final=pd.read_csv('../Volvo/prediction_sample.csv')
Final.pred=PredOut.final

In [13]:
Final

,pred
0,Low
1,Low
2,Low
3,Low
4,Low
...,...
33585,High
33586,High
33587,High
33588,High


In [12]:
Final.to_csv('prediction_.csv')